<a href="https://colab.research.google.com/github/rjnakano/manejo-inventarios-pdg/blob/main/Notebooks/Silueta_bootstrapV1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install --upgrade kmodes

Requirement already up-to-date: kmodes in /usr/local/lib/python3.7/dist-packages (0.11.0)


In [ ]:
from google.colab import drive
import pathlib
import pandas as pd
import numpy as np
from pylab import *

from sklearn.preprocessing import StandardScaler

from kmodes.kprototypes import KPrototypes

import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx

import seaborn as sns
%matplotlib inline
sns.set_context('poster')
sns.set_style('white')
sns.set_color_codes()

from plotnine import *
import plotnine

In [ ]:
plot_kwds = {'alpha' : 0.5, 's' : 1, 'linewidths':0}

In [ ]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# Dataset de inventarios sin dimensiones de los productos
data1 = pd.read_csv("/content/gdrive/MyDrive/TDGdata/df042421.csv")

In [ ]:
data1['AISLE']=data1['AISLE'].astype(str)

In [ ]:
data1.dtypes

AISLE        object
IG           object
HTS          object
PLANNING     object
ABC          object
UOM          object
CAT          object
COSTX       float64
QTYVAR      float64
dtype: object

In [ ]:
data1.head(5)

,AISLE,IG,HTS,PLANNING,ABC,UOM,CAT,COSTX,QTYVAR
0,17,IG56,HTS119,1/1,B,PC,CAT17,-0.029671,-0.037167
1,17,IG56,HTS119,1/1,B,PC,CAT17,0.092655,-0.006905
2,17,IG56,HTS119,1/1,C,PC,CAT17,0.266138,0.020477
3,17,IG56,HTS119,1/1,B,PC,CAT17,-0.024812,-0.044242
4,15,IG56,HTS119,1/1,C,PC,CAT17,-0.077628,-0.046455


In [ ]:
# scikit-learn bootstrap
from sklearn.utils import resample

In [ ]:
# Matriz de distancias
dm = pd.read_csv("/content/gdrive/MyDrive/TDGdata/distancias.csv")
dm=dm.to_numpy()

In [ ]:
import time
start_time = time.time()
# Coeficiente silueta para K 2-4
kValues = [2, 3, 4]
gamma = 0.5
nBootstrap = 10
# Inicializamos la matriz de coeficientes silueta con ceros
silueta = [ [ 0 for j in range(nBootstrap + 1) ] for i in range(len(kValues)) ]
catColumnsPos = [data1.columns.get_loc(col) for col in list(data1.select_dtypes('object').columns)]
numColumnsPos = [data1.columns.get_loc(col) for col in list(data1.select_dtypes('float64').columns)]
N = data1.shape[0]

In [ ]:
from sklearn.metrics import silhouette_score
for k in kValues:
  print('k = ', k)
  kprototype = KPrototypes(n_jobs = -1, n_clusters = k, init = 'Huang', gamma = gamma, random_state = 0)
  kprototype.fit_predict(data1, categorical = catColumnsPos)
  print("--- %s seconds ---" % (time.time() - start_time))
  silueta[k-min(kValues)][0] = silhouette_score(dm, kprototype.labels_, metric='precomputed')
  print("Coeficiente silueta --- %s seconds ---" % (time.time() - start_time))
  print(silueta[k-min(kValues)][0])
  for b in range(1,nBootstrap+1):
    print('bootstrap # ',b);
    boot = resample(data1, replace=True, n_samples=data1.shape[0], random_state=b)
    # Fit the cluster
    kprototypebs = KPrototypes(n_jobs = -1, n_clusters = k, init = 'Huang', gamma = gamma, random_state = 1)
    kprototypebs.fit_predict(boot, categorical = catColumnsPos)
    print("--- %s seconds ---" % (time.time() - start_time));
    # boot dividido en atributos numericas y categoricos
    dataNum=boot.iloc[:,numColumnsPos]
    dataNum=dataNum.to_numpy()
    dataCat=boot.iloc[:,catColumnsPos]
    dataCat=dataCat.to_numpy()
    # Inicializamos la matriz de distancias con ceros
    scores = [ [ 0 for i in range(N) ] for j in range(N) ]
    # Calculamos la mitad de la matriz distancias
    for i in range(N):
      for j in range(i+1,N):
        scores[i][j]=np.sum((dataNum[i] - dataNum[j]) ** 2) + gamma * np.sum(dataCat[i]!=dataCat[j])
    print("Matriz de distancias --- %s seconds ---" % (time.time() - start_time))
    # Completamos la matriz
    for i in range(0,N):
      for j in range(0,i):
        scores[i][j]=scores[j][i]
    print("Espejo --- %s seconds ---" % (time.time() - start_time))
    silueta[k-min(kValues)][b] = silhouette_score(scores, kprototypebs.labels_, metric='precomputed')
    print("Coeficiente silueta --- %s seconds ---" % (time.time() - start_time))
    print(silueta[k-min(kValues)][b])

k =  2
--- 57.606810092926025 seconds ---
Coeficiente silueta --- 60.2447350025177 seconds ---
0.736586459830793
bootstrap #  1
--- 99.95554876327515 seconds ---
Matriz de distancias --- 2248.4654326438904 seconds ---
Espejo --- 2304.2878198623657 seconds ---
Coeficiente silueta --- 2345.1558418273926 seconds ---
0.8101794559566166
bootstrap #  2
--- 2389.026867866516 seconds ---
Matriz de distancias --- 4522.625143527985 seconds ---
Espejo --- 4575.12732052803 seconds ---
Coeficiente silueta --- 4617.210820436478 seconds ---
0.7336153098017565
bootstrap #  3
--- 4660.934317111969 seconds ---


KeyboardInterrupt: ignored

In [ ]:
data1['Cluster'] = kprototype.labels_

In [ ]:
silueta

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [ ]:
for i in kValues:
  print('Mediana para k = ',i);
  print(median(silueta[i-min(kValues)][1:nBootstrap+1]));

Mediana para k =  2
0.0
Mediana para k =  3
0.0
Mediana para k =  4
0.0


In [ ]:
np.savetxt("/content/gdrive/MyDrive/TDGdata/siluetaboot.csv", silueta, delimiter=",")

In [ ]:
silueta = pd.DataFrame(silueta)

In [ ]:
silueta.describe()